# Breaking Bad : Web scrapping summaries from [Fandom.com](https://breakingbad.fandom.com/wiki/Breaking_Bad_(TV_series))
<img src = 'src/imgs/Breaking_Bad_Poster1.jpg' width = 50%>


### Series Summary

- **Breaking Bad** is an American crime drama television series created and produced by Vince Gilligan.
- Set and filmed in Albuquerque, New Mexico, the series follows Walter White (Bryan Cranston), an underpaid, overqualified, and dispirited high-school chemistry teacher who is struggling with a recent diagnosis of stage-three lung cancer.
- Walt turns to a life of crime and partners with a former student, Jesse Pinkman (Aaron Paul), to produce and distribute crystal meth to secure his family's financial future before he dies, while navigating the dangers of the criminal underworld.
- The show aired on AMC from January 20, 2008, to September 29, 2013, consisting of five seasons for a total of 62 episodes.

### The Plan

- Will generate link to each episode in the series.
- Once linkes are populated, 'll iterate over it to get, 
    - Character names and
    - episode summary.
- Combine each seasons episode summary into one file.
- Save the summary of each summary in separate files,
    - ie for 6 seasons, we'll have total 6 summary files.

# 1. Importing libraries

In [1]:
import os
import re
import time

import shutil
import requests

import numpy as np
import pandas as pd
from tqdm import tqdm

from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.core.utils import ChromeType
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromiumService

In [2]:
# Ensuring empty directory is created

if os.path.isdir('data'):
    shutil.rmtree('data')
os.makedirs('data/summaries')

# 2. Getting started with scrapping

In [3]:
# Parsing website skelton to bs4 object

BASE_URL = 'https://breakingbad.fandom.com' 

r = requests.get(BASE_URL + '/wiki/Category:Seasons_(Breaking_Bad)')

soup = bs(r.content, 'lxml')

# 3. Generating links of seasons main page

In [4]:
# Generating links to each seasons web page

all_items_S = soup.find_all('div', class_ = 'category-page__members-wrapper')[-1]
season_list = all_items_S.find_all('li', class_ = 'category-page__member')

season_link = []

for season in season_list:
    season_link.append((season.find('a').get('title'), BASE_URL + season.find('a').get('href')))

for sea, link in season_link:
    print(f'{sea} :: {link}')

Season 1 (Breaking Bad) :: https://breakingbad.fandom.com/wiki/Season_1_(Breaking_Bad)
Season 2 (Breaking Bad) :: https://breakingbad.fandom.com/wiki/Season_2_(Breaking_Bad)
Season 3 (Breaking Bad) :: https://breakingbad.fandom.com/wiki/Season_3_(Breaking_Bad)
Season 4 (Breaking Bad) :: https://breakingbad.fandom.com/wiki/Season_4_(Breaking_Bad)
Season 5A (Breaking Bad) :: https://breakingbad.fandom.com/wiki/Season_5A_(Breaking_Bad)
Season 5B (Breaking Bad) :: https://breakingbad.fandom.com/wiki/Season_5B_(Breaking_Bad)


# 4. Defining helper functions

In [5]:
# https://pypi.org/project/webdriver-manager/

def call_webdriver():
    
    return webdriver.Chrome(service = ChromiumService(ChromeDriverManager(chrome_type = ChromeType.CHROMIUM).install()))

In [6]:
def get_summmary(episode_link, season_name):
    
    driver = call_webdriver()
    driver.minimize_window()
    driver.get(episode_link)
    time.sleep(5)
    all_para = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/h3[1]//following::p')
    
    with open(f'data/summaries/{season_name}.txt', 'a') as file:
        for para in all_para:
            if para.aria_role =='paragraph':
                file.write(para.text + '\n')
                
    # list_para = [para.text for para in all_para if para.aria_role =='paragraph']
    driver.close()
    # return list_para

In [7]:
def get_starring(episode_link):
    
    r = requests.get(episode_link)
    soup = bs(r.content, 'lxml')
    
    starring_block = soup.find('div', class_ = 'tabber wds-tabber')
    charactres_class = starring_block.find_all('div', class_ = 'wds-tab__content')

    # for char in charactres_class:
    #     for ch in char.select('li'):
    #         print(ch.text)
        
    starrings = [ch.text for char in charactres_class for ch in char.select('li')]
        
    # starrings = [star.text for star in soup.select('div.tabber.wds-tabber table li')]
    
    # for char in starrings:
    #     print(char.text)
    
    return starrings

# 5. Gathering each episodes summary and character names

In [8]:
episode_links, characters = {}, {}

for season, link in tqdm(season_link):
    
    r = requests.get(link)

    soup = bs(r.content, 'lxml')
    episodes = ['https://breakingbad.fandom.com' + x.find('a').get('href') for x in soup.select('#gallery-0 div.thumb')]
    
    season = season.split(' (')[0].replace(' ', '_')
    episode_links[season] = episodes
    
    characts = []
    
    for episode in episodes:
        get_summmary(episode, season)
        characts.append(get_starring(episode))
        
    characters[season] = list(np.concatenate(characts))
    
    # cnt += len(episodes)
    # break
        
print(f'Number of episodes recoreded   :: {np.concatenate(list(episode_links.values())).shape[0]}')

100%|███████████████████████████████████████████████████████████████| 6/6 [1:20:17<00:00, 802.86s/it]

Number of episodes recoreded   :: 62


# 6. Processing obtained character name info.

In [9]:
# Saving episodes link to a text file for future needs

with open('data/season_nd_episode_links.txt', 'w') as file:
        file.write(str(episode_links))

In [10]:
Season_1 = ['Season_1'] * len(characters['Season_1'])
Season_2 = ['Season_2'] * len(characters['Season_2'])
Season_3 = ['Season_3'] * len(characters['Season_3'])
Season_4 = ['Season_4'] * len(characters['Season_4'])
Season_5A = ['Season_5A'] * len(characters['Season_5A'])
Season_5B = ['Season_5B'] * len(characters['Season_5B'])

seas = Season_1 + Season_2 + Season_3 + Season_4 + Season_5A + Season_5B

chars = characters['Season_1'] + characters['Season_2'] + characters['Season_3'] + characters['Season_4'] + characters['Season_5A'] + characters['Season_5B']

In [11]:
character_df = pd.DataFrame({'Season' : seas, 'Characters' : chars})
character_df.to_csv('data/character_df.csv', index = False)
character_df.head()

,Season,Characters
0,Season_1,Bryan Cranston as Walter White
1,Season_1,Anna Gunn as Skyler White
2,Season_1,Aaron Paul as Jesse Pinkman
3,Season_1,Dean Norris as Hank Schrader
4,Season_1,Betsy Brandt as Marie Schrader


In [12]:
print('Number of Characters recorded per season'); print('-' * 40)
character_df.Season.value_counts(sort = False)

Number of Characters recorded per season
----------------------------------------


Season_1     133
Season_2     234
Season_3     264
Season_4     270
Season_5A    157
Season_5B    195
Name: Season, dtype: int64

In [13]:
# Cleaning character_df : Removing and stripping unwanted characters

character_df.Characters = character_df.Characters.apply(lambda x : re.sub(u'\xa0', u' ', x).strip()) # https://stackoverflow.com/a/11566398
character_df.Characters = character_df.Characters.apply(lambda x : re.sub(r'\t', ' ', x).strip())
character_df.Characters = character_df.Characters.apply(lambda x : x.split(' as ')[-1].strip())
character_df.Characters = character_df.Characters.apply(lambda x : re.sub(r'^as\s', ' ', x).strip())

In [14]:
for idx in [19, 20, 32, 1230]:
    print(character_df.iloc[idx][1])

Jock's Friend #1
Jock's Friend #2
Hank Schrader (credit only)
Hank Schrader (archive footage)


In [15]:
# Processing character names

character_df.Characters = character_df.Characters.apply(lambda x : re.sub(r'#\d*', ' ', x).strip()) # Removing '#' from name
character_df.Characters = character_df.Characters.apply(lambda x : re.sub(r'\(.*\)', ' ', x).strip()) # Removing '()' and text inside those
character_df.Characters = character_df.Characters.apply(lambda x : re.sub(r'CarWash Patron', 'Car Wash Patron', x).strip())
character_df.Characters = character_df.Characters.apply(lambda x : re.sub('\s+', ' ', x).strip()) # Removing unwanted extra white spaces

In [16]:
print(f'Total number of characters\t\t:: {len(character_df.Characters)}')
print(f'Total number of unique characters\t:: {character_df.Characters.nunique()}')

Total number of characters		:: 1253
Total number of unique characters	:: 385


In [17]:
character_df.head()

,Season,Characters
0,Season_1,Walter White
1,Season_1,Skyler White
2,Season_1,Jesse Pinkman
3,Season_1,Hank Schrader
4,Season_1,Marie Schrader


In [18]:
character_df.to_csv('data/character_df_cleaned.csv', index = False)
character_df.tail()

,Season,Characters
1248,Season_5B,Albuquerque Police Officer
1249,Season_5B,DEA
1250,Season_5B,Steven Gomez
1251,Season_5B,Jack's Henchman
1252,Season_5B,Student in bus


# 7. Next is What ?

- We have generated summaries all seasons using web scrapping in this part.
- In the next part file [relationship_finder.ipynb](https://github.com/jishnukoliyadan/the_breaking_bad_network/blob/master/relationship_finder.ipynb), we will try to find the relationsship between characters using `spaCy` and `NetworkX` libraries.

# References

References
- Official documentations of all libraries
- [Fandom](https://www.fandom.com/)
- [GitHub : scrapper_github_topics](https://github.com/jishnukoliyadan/scrapper_github_topics)
- [DataCamp - A Network Analysis of Game of Thrones](https://www.datacamp.com/projects/76)

Query Resolution
- [GeeksforGeeks](https://www.geeksforgeeks.org/)
- [Stack Overflow](https://stackoverflow.com/)
    
Image Credits
- [IMDb.com](https://www.imdb.com/)